In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Lambda, Compose, Normalize
from collections import defaultdict
from torch.utils.data import random_split
import copy

from NN_utils import *
from NN_utils.train_and_eval import *
from uncertainty import train_NN_with_g
from uncertainty.losses import penalized_uncertainty
import uncertainty.comparison as unc_comp
import uncertainty.quantifications as unc


from torch.utils.data import random_split

In [2]:
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


In [3]:
transforms_train = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
transforms_ = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

In [4]:
training_data = datasets.CIFAR10(
root="data",
 train=True,
 download=True,
transform=transforms_train)

test_data = datasets.CIFAR10(
root="data",
train=False,
download=True,
transform=transforms_)

train_size = int(0.5*len(training_data))
val_size = len(training_data) - train_size
training_data, validation_data = random_split(training_data, [train_size, val_size])

Files already downloaded and verified
Files already downloaded and verified


In [5]:
batch_size = 12
train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle = True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size,shuffle = True)
test_dataloader = DataLoader(test_data, batch_size=12)

In [6]:
from NN_models.CIFAR10 import Model_CNN_10_with_g as Model_CNN_with_g

In [10]:
class Trainer(torch.nn.Module):
    def __init__(self,model,optimizer,loss_criterion, print_loss = True,keep_hist = True):
        # adaptar keep hist para definir oq manter\n",
        super().__init__()
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_criterion
        self.print_loss = print_loss
        self.keep_hist = keep_hist

        self.g_list = []
        self.acc_list = []
        self.bce_list = []
        self.bce_iter = []
        self.acc_c_g = []
        self.acc_c_mcp = []
        
    def fit(self,data,n_epochs):
        for epoch in range(1,n_epochs+1):
            self.train_NN_with_g(data,1,set_train_mode = True)
            if self.keep_hist:
                self.update_hist(data)
            self.loss_fn.update_L0(self.bce_list[-1])
            #self.bce_iter = []

    def fit_x(self,data,n_epochs,ignored_layers = ['fc_g_layer']):
        loss_criterion = copy.copy(self.loss_fn.criterion)
        loss_criterion.reduction = 'mean'
        ignore_layers(self.model,ignored_layers,reset = True)
        for epoch in range(1,n_epochs+1):
            train_NN(self.model,self.optimizer,data,loss_criterion,1, self.print_loss, set_train_mode = True)
            if self.keep_hist:
                self.update_hist(data)
                ignore_layers(self.model,ignored_layers,reset = True)

    def fit_g(self,data,n_epochs,ignored_layers = ['conv_layer','fc_layer','fc_x_layer']):
        
        _,_,bce = model_metrics(self.model,nn.NLLLoss(),data)
        self.loss_fn.update_L0(bce)#(self.bce_list[-1])
        ignore_layers(self.model,ignored_layers, reset = True)
        for epoch in range(1,n_epochs+1):
            self.train_NN_with_g(data,1, set_train_mode = False)
            if self.keep_hist:
                self.update_hist(data)
                ignore_layers(self.model,ignored_layers, reset = True)
        
            
    def train_NN_with_g(self,data,n_epochs, set_train_mode = True):
            '''Train a NN that has a g layer'''
            dev = next(self.model.parameters()).device
            if set_train_mode:
                self.model.train()
            for epoch in range(n_epochs):
                running_loss = 0
                for image,label in data:
                    image,label = image.to(dev), label.to(dev)
                    self.optimizer.zero_grad()
                    output = self.model(image)
                    g = self.model.get_g()
                    loss = self.loss_fn(output,g,label)
                    self.bce_iter.append(torch.mean(self.loss_fn.criterion(output,label)).item())
                    self.loss_fn.update_L0(np.mean(self.bce_iter[-2080:]))
                    #_, _, bce =  model_metrics(self.model,self.loss_fn.criterion,data)
                    #self.loss_fn.update_L0(bce)
                    
                    loss.backward()
                    self.optimizer.step()
                    running_loss += loss.item()
                if self.print_loss:
                    print('Epoch ', epoch+1, ', loss = ', running_loss/len(data))
                    
    def update_hist(self,data):
        with torch.no_grad():
            loss_criterion = copy.copy(self.loss_fn.criterion)
            loss_criterion.reduction = 'mean'
            #acc, g, bce =  model_metrics(self.model,loss_criterion,data)
            #self.g_list.append(g)
            #self.acc_list.append(acc)
            #self.bce_list.append(bce)

            label, output, g = accumulate_results(self.model,data)
            bce = loss_criterion(output,label).item()
            output = torch.exp(output)
            mcp = unc.MCP_unc(output)

            self.acc_c_g.append(unc_comp.acc_coverage(output,label,1-g,0.2))
            self.acc_c_mcp.append(unc_comp.acc_coverage(output,label,mcp,0.2))

            self.g_list.append(torch.mean(g).item())
            acc = correct_total(output,label)/label.size(0)
            self.acc_list.append(acc)
            self.bce_list.append(bce)

In [11]:
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

In [13]:

model_3_2 = Model_CNN_with_g()
model_3_2 = model_3_2.to(dev)
optimizer = torch.optim.SGD(model_3_2.parameters(), lr=1e-3)
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_3_2 = Trainer(model_3_2,optimizer,loss_fn, print_loss = True,keep_hist = True)
model_trainer_3_2.fit_x(train_dataloader,40)
model_trainer_3_2.fit_g(validation_dataloader,40)

acc, g, bce = model_metrics(model_3_2,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_3_2,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

Epoch  1 , loss =  2.2426244883642545
Epoch  1 , loss =  1.9744338378910826
Epoch  1 , loss =  1.771480081577905
Epoch  1 , loss =  1.6492122118280854
Epoch  1 , loss =  1.5608549471837316
Epoch  1 , loss =  1.4859781486631125
Epoch  1 , loss =  1.4065913258266998
Epoch  1 , loss =  1.345102991601327
Epoch  1 , loss =  1.2847553878850992
Epoch  1 , loss =  1.218910352453847
Epoch  1 , loss =  1.162887529682747
Epoch  1 , loss =  1.1157685301754616
Epoch  1 , loss =  1.0640397767988596
Epoch  1 , loss =  1.0298798702900331
Epoch  1 , loss =  0.9849911584534938
Epoch  1 , loss =  0.9583527671312645
Epoch  1 , loss =  0.9367534164661067
Epoch  1 , loss =  0.9061683203002542
Epoch  1 , loss =  0.8837059345060598
Epoch  1 , loss =  0.8583660617599446
Epoch  1 , loss =  0.8406986635385686
Epoch  1 , loss =  0.8133808310839014
Epoch  1 , loss =  0.8010108187051057
Epoch  1 , loss =  0.7811012306463352
Epoch  1 , loss =  0.7584868811263976
Epoch  1 , loss =  0.7486157978092983
Epoch  1 , loss 

NameError: name 'model' is not defined